In [9]:
!pip install python-keycloak
!pip install psycopg2-binary
import csv
import requests
import psycopg2
from datetime import datetime
from dateutil import parser
from dateutil import parser
from keycloak import KeycloakOpenID
from keycloak import KeycloakAdmin
import keycloak
import json
 
CPU_HRS_MONTH = "dec"
CPU_HRS_YEAR = 2021
POSTGRES_USER = "postgres"
POSTGRES_PASSWORD = "admin1234"
POSTGRES_HOST = "mosaicai-sandbox.catnchrgoqwp.us-east-1.rds.amazonaws.com"
POSTGRES_PORT = "5432"
POSTGRES_DB_CONSOLE = "mosaic_console"
POSTGRES_DB_AI = "ai_logistics"
POSTGRES_DB_METERING = "metering"
POSTGRES_DB_USERMGM = "usermgm"
SERVER_URL="http://mosaic-auth-249051266.us-east-1.elb.amazonaws.com/auth/"
REALM="master"
CLIENT_ID="admin-cli"
KEYCLOAK_USERNAME="keycloak-adm"
KEYCLOAK_PASSWORD="90f8372d933d4fd1fd311f0b8cd36a3d"

keycloak_openid = KeycloakOpenID(server_url=SERVER_URL,
                        client_id=CLIENT_ID,
                        realm_name=REALM,                                       
                        client_secret_key="")

token = keycloak_openid.token(KEYCLOAK_USERNAME, KEYCLOAK_PASSWORD)
access_token = token['access_token']
def get_user_activity_details(access_token):
    try:
        connection_usermgm = psycopg2.connect(user=POSTGRES_USER,
                                              password=POSTGRES_PASSWORD,
                                              host=POSTGRES_HOST,
                                              port=POSTGRES_PORT,
                                              database=POSTGRES_DB_USERMGM,
                                              options="-csearch_path=usermgm")
        result = getUsersActivity(access_token, connection_usermgm)
        return result
    except Exception as ex:
        print("Exception : ", ex)
        
def get_all_users_keycloak(tokenn):
    #print(tokenn)
    url=SERVER_URL+"admin/realms/mosaic/users?briefRepresentation=true&first=0&max=20"
    auth_token = "Bearer "+tokenn
    headers= {"Authorization": auth_token, "accept": "application/json" }
    response = requests.get(url, headers=headers)
    return response.json()

def get_all_users():
    url="http://user-management:8094/usermgm/user/getAllUsers"
    headers= {"X-Auth-Userid": "mosaic", "accept": "application/json"}
    response = requests.get(url, headers=headers)
    return response.json()

def get_users_representation(tokenn):
    #print(tokenn)
    url=SERVER_URL+"admin/realms/mosaic/users"
    auth_token = "Bearer "+tokenn
    headers= {"Authorization": auth_token, "accept": "application/json" }
    response = requests.get(url, headers=headers)
    return response.json()

def get_users_events(userId, startDateStr, tokenn):
    url=SERVER_URL+"admin/realms/mosaic/events?type=LOGIN"
    if startDateStr!="":
        url=SERVER_URL+"admin/realms/mosaic/events?dateTo="+startDateStr+"&user="+userId
    auth_token = "Bearer "+tokenn
    headers= {"Authorization": auth_token, "accept": "application/json" }
    response = requests.get(url, headers=headers)
    return response.json()

def createUsersInfo(userRepresentations, usersList, allUsers):
    userInfoList = []
    for userRepresentation in userRepresentations:
        usersListStr = ' '.join([str(elem) for elem in usersList])
        if userRepresentation['id'] in usersListStr:
            userMap = {}
            userMap["username"] = userRepresentation['username']
            userMap["email"]= userRepresentation['email'];
            userMap["name"]= userRepresentation['firstName'] + userRepresentation['lastName'];
            userMap["id"] = userRepresentation['id'];
            for user in allUsers:
                if ((user['username']).casefold()== (userRepresentation['username']).casefold()):
                    userMap["group"]= ', '.join(user["groups"])
            userInfoList.append(userMap);
    return userInfoList

def getUsersActivity(token, connection_usermgm):
    eventsList = ["LOGIN", "LOGOUT"]
    usersList = set()
    eventUsers= set()
    activeUsersList = []
    responseList =[]
    userRepresentations = get_users_representation(token)
    for userRepresentation in userRepresentations:
        usersList.add(userRepresentation['id'])
    allUsers = get_all_users()
    userInfoList = createUsersInfo(userRepresentations, usersList, allUsers)
    startDateStr = (datetime.combine(datetime.now().date(), datetime.min.time())).strftime('%Y-%m-%d')
    eventRepresentationsAllUsers = get_users_events("", "", token)
    for event in eventRepresentationsAllUsers:
        eventUsers.add(event['userId'])
    roleRequests = get_user_role_requests(connection_usermgm)
    inactive_users = [x for x in usersList if x not in eventUsers]
    
    for userId in eventUsers:        
        eventRepresentations = get_users_events(userId, startDateStr, token)
        activeUsersMap = {}
        activeUsersMap['id'] = userId
        lasloginTime = eventRepresentations[0]["time"]
        firstLoginTime = eventRepresentations[len(eventRepresentations)-1]["time"]            
        activeUsersMap['lastLoginTime']=lasloginTime
        activeUsersMap['firstLoginTime']=firstLoginTime
        activeUsersMap['loginCount']=len(eventRepresentations)
        activeUsersList.append(activeUsersMap)
        responseList = buildUserActivityList(userInfoList, activeUsersMap, responseList, None)
    for inactive_user in inactive_users:
        responseList = buildUserActivityList(userInfoList, {}, responseList, inactive_user)
    for response in responseList:
        team = set()
        country = set()
        model_family = set()
        for roleRequest in roleRequests:
            if (roleRequest[3]).casefold()==(response["username"]).casefold() :
                team.add(roleRequest[20])
                country.add(roleRequest[9])
                model_family.add(roleRequest[19])
        response["Team"]= ', '.join(team)
        response["Country"]= ', '.join(country)
            #response["Model_family"] = model_family
    return responseList
    
def buildUserActivityList(userInfoList, activeUser, responseList, inactive_user):
    for userInfo in userInfoList:
        if len(activeUser) > 0 and userInfo['id'] == activeUser['id']:
            dt_object = datetime.fromtimestamp(int(str(activeUser['lastLoginTime'])[:-3]))
            userInfo['lastLoginTime']= str(dt_object)
            dt_object = datetime.fromtimestamp(int(str(activeUser['firstLoginTime'])[:-3])) 
            userInfo['firstLoginTime']= str(dt_object)
            userInfo['loginCount']= activeUser['loginCount']
            responseList.append(userInfo)
        elif userInfo['id'] == inactive_user:
            userInfo['lastLoginTime']= ""
            userInfo['firstLoginTime']= ""
            userInfo['loginCount']= 0
            responseList.append(userInfo)
    return responseList

def get_user_role_requests(connection):
    try:
        cursor = connection.cursor()
        postgreSQL_select_Query = "select * from user_role_request where dm_request_status=1"
        cursor.execute(postgreSQL_select_Query)
        user_role_requests = cursor.fetchall()
        return user_role_requests
    except (Exception, psycopg2.Error) as error:
        print("Error while fetching data from PostgreSQL for : ", error)
        return -1
    finally:
        cursor.close()

#=====================Template============================
def get_all_users():
    url="http://user-management:8094/usermgm/user/getAllUsers"
    headers= {"X-Auth-Userid": "mosaic", "accept": "application/json"}
    response = requests.get(url, headers=headers)
    return response.json()

def get_total_projects_for_given_user(username):
    url="http://mosaic-console-backend:80/mosaic-console-backend/secured/api/project/v1"
    headers= {"X-Auth-Userid": username,"X-Auth-Username": username, "accept": "application/json"}
    response = requests.get(url, headers=headers)
    return response.json()

def get_the_total_templates_and_notebook_runs(username):
    payload = {
        "userList":[username],
        "recordCount": 1000000,
        "pageNo":1,
        "componentType":[]
    }
    url="http://mosaic-console-backend:80/mosaic-console-backend/secured/api/activity/v1/filter"
    headers= {"X-Auth-Userid": username,"X-Auth-Username": username, "accept": "application/json"}
    response = requests.post(url, headers=headers, json=payload)
    response_json = response.json()["data"]
#     if username == 'mosaic':
#         print(response_json)
    notebook_runs = len([x for x in response_json if x["object_type"] == 'NOTEBOOK' and x["action"] == 'RUN'])
    template_create = len([x for x in response_json if x["object_type"] == 'TEMPLATE' and x["action"] == 'CREATE'])
    return {"notebook_runs":notebook_runs, "template_create":template_create}


def get_project_game_linkage_count(connection, username):
    try:
        cursor = connection.cursor()
        postgreSQL_select_Query = "select count(1) as project_count from game_project_master where created_by = '"+username+"'"
        cursor.execute(postgreSQL_select_Query)
        no_of_notebooks = cursor.fetchone()
        return no_of_notebooks[0]
    except (Exception, psycopg2.Error) as error:
        print("Error while fetching data from PostgreSQL for '"+username+"' : ", error)
        return -1
    finally:
        cursor.close()
        
        
def get_sas_template_count(connection, username):
    try:
        cursor = connection.cursor()
        postgreSQL_select_Query = "SELECT count(1) AS sas_template_spwan_cnt FROM nb_template_status t_status INNER JOIN nb_docker_image template ON TRIM(t_status.template_id) = TRIM(template.id) WHERE t_status.created_by='"+username+"' AND template.base_image_id=(select id from nb_docker_image where name='SAS')"
        cursor.execute(postgreSQL_select_Query)
        no_of_notebooks = cursor.fetchone()
        return no_of_notebooks[0]
    except (Exception, psycopg2.Error) as error:
        print("Error while fetching data from PostgreSQL for '"+username+"' : ", error)
        return -1
    finally:
        cursor.close()
        

def get_cpu_hrs_per_user(connection, username):
    try:
        cursor = connection.cursor()
        postgreSQL_select_Query = "select SUM(CAST(current_usage->>'cpu_hours' AS FLOAT)) from subscribers  where created_by='"+username+"' group by created_by"
        cursor.execute(postgreSQL_select_Query)
        no_of_notebooks = cursor.fetchone()
        return no_of_notebooks[0]
    except (Exception, psycopg2.Error) as error:
        #print("Error while fetching data from PostgreSQL for '"+username+"' : ", error)
        return 0
    finally:
        cursor.close()

Processing /home/mosaic-ai/.cache/pip/wheels/16/1b/c2/e2b4a69796a48c66ed60a04004282fec031d1a1f321eece7c2/python_keycloak-0.26.1-cp37-none-any.whl
  Using cached https://files.pythonhosted.org/packages/2d/61/08076519c80041bc0ffa1a8af0cbd3bf3e2b62af10435d269a9d0f40564d/requests-2.27.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bd/2d/e94b2f7bab6773c70efc70a61d66e312e1febccd9e0db6b9e0adf58cbad1/python_jose-3.3.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/84/3e/1037abe6498e65d645ce7a22d3402605d49a3b2c7f20c3abb027760da4f0/charset_normalizer-2.0.10-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4e/b8/f5a25b22e803f0578e668daa33ba3701bb37858ec80e08a150bd7d2cf1b1/urllib3-1.26.8-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/37/45/946c02767aabb873146011e665728b680884cd8fe70dde973c640e45b775/certifi-2021.10.8-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/pack

In [10]:
try:
    report_records = []
    #all_users = get_all_users()
    all_users = get_user_activity_details(access_token)
    print("The total no of users are : ", len(all_users))
    connection_mosaic_console = psycopg2.connect(user=POSTGRES_USER,
                                                 password=POSTGRES_PASSWORD,
                                                 host=POSTGRES_HOST,
                                                 port=POSTGRES_PORT,
                                                 database=POSTGRES_DB_CONSOLE)
    connection_ai_logistics = psycopg2.connect(user=POSTGRES_USER,
                                                 password=POSTGRES_PASSWORD,
                                                 host=POSTGRES_HOST,
                                                 port=POSTGRES_PORT,
                                                 database=POSTGRES_DB_AI)
    connection_metering = psycopg2.connect(user=POSTGRES_USER,
                                                 password=POSTGRES_PASSWORD,
                                                 host=POSTGRES_HOST,
                                                 port=POSTGRES_PORT,
                                                 database=POSTGRES_DB_METERING)
    for user in all_users:
        report_record = {}
        username = user["username"]
        report_record['USER_NAME']=username
        report_record['LAST_LOGIN']=user["lastLoginTime"]
        report_record['FIRST_LOGIN']=user["firstLoginTime"]
        report_record['LOGIN_COUNT']=user["loginCount"]
        report_record['TEAM']=user["Team"]
        report_record['COUNTRY']=user["Country"]
        #print("\n")
        #print("Username :", username)
    
        # TOTAL PROJECTS 
        all_projects = get_total_projects_for_given_user(username)
        report_record["TOTAL_PROJECTS_CREATED"]=len(all_projects)
        #print("Total Projects : "+str(len(all_projects)))
    
        # TOTAL TEMPLATES & NOTEBOOKS RUN
        template_details = get_the_total_templates_and_notebook_runs(username)
        report_record["TOTAL_TEMPLATES_CREATED"] = template_details["template_create"]
        #print("Total Templates Created : "+str(template_details["template_create"]))
        report_record["TOTAL_NOTEBOOKS_SPAWNED"] = template_details["notebook_runs"]
        #print("Total Notebooks Runs : "+str(template_details["notebook_runs"]))
    
        #CPU Hrs
        total_cpu_hrs = get_cpu_hrs_per_user(connection_metering, username) 
        report_record["CPU_Hrs"] = total_cpu_hrs
        #print("The total CPU Hrs are : ", total_cpu_hrs)
        
        # Project Game Linkage
        project_game_linkage_count = get_project_game_linkage_count(connection_mosaic_console, username)   
        report_record["TOTAL_PROJECTS_LINKED_TO_GAME"] = project_game_linkage_count
        #print("Project-Game Linkage Count: ", project_game_linkage_count)
                                               
        # SAS Template Count
        sas_template_count = get_sas_template_count(connection_ai_logistics, username)  
        report_record["TOTAL_SAS_TEMPLATES_SPAWNED"] = sas_template_count
        #print("Total SAS Template Count: ", sas_template_count)
        
        report_records.append(report_record)
        
    if report_records:
        with open('/data/adoption_matrix_report_userlevel.csv', 'w', newline='') as output_file:
            dict_writer = csv.DictWriter(output_file, report_records[0].keys())
            dict_writer.writeheader()
            dict_writer.writerows(report_records)
    
    print("DONE !!")
except Exception as ex:
      print("Exception : ", ex)
finally:
    if connection_mosaic_console:
        connection_mosaic_console.close()
        print("\nPostgres Connection 'connection_mosaic_console' is closed") 
    if connection_ai_logistics:
        connection_ai_logistics.close()
        print("\nPostgres Connection 'connection_ai_logistics' is closed") 
    if connection_metering:
        connection_metering.close()
        print("\nPostgres Connection 'connection_metering' is closed")

The total no of users are :  6
DONE !!

Postgres Connection 'connection_mosaic_console' is closed

Postgres Connection 'connection_ai_logistics' is closed

Postgres Connection 'connection_metering' is closed
